In [1]:
from google.colab import files
uploaded=files.upload()

Saving kaggle.json to kaggle.json


In [2]:
!pwd

/content


In [3]:
import os
os.environ['KAGGLE_CONFIG_DIR']='/content'

In [4]:
!kaggle datasets download -d alanchn31/free-spoken-digits --force

 70% 11.0M/15.7M [00:00<00:00, 24.8MB/s]
100% 15.7M/15.7M [00:00<00:00, 30.0MB/s]


In [1]:
from fastai.vision.all import *
from fastcore.all import *
import shutil
import matplotlib.pyplot as plt
%matplotlib inline
import librosa
import librosa.display
from tqdm import tqdm
import os
import zipfile
from pathlib import Path
import wave
import numpy as np
import IPython
from IPython.display import Image, display

In [ ]:
local_zip = '/content/free-spoken-digits.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall()
zip_ref.close()

In [ ]:
#Converting wav files into png format by making spectrogram for each audio file and saving them into drive 
#Audio files are divided into 10 classes from 0 to 9
import os
import scipy.io.wavfile as wav
import wave
output_dir='/content/drive/MyDrive/spectrograms_2'
data_dir='/content/free-spoken-digit-dataset-master/recordings'
for filename in os.listdir(data_dir):
                 if "wav" in filename:
                    file_path = os.path.join(data_dir, filename)
                    target_dir = f'class_{filename[0]}'             
                    dist_dir = os.path.join(output_dir, target_dir)
                    file_dist_path = os.path.join(dist_dir, filename)
                    if not os.path.exists(file_dist_path + '.png'):
                        if not os.path.exists(dist_dir):
                            os.mkdir(dist_dir)                
                        frame_rate, data = wav.read(file_path)
                        signal_wave = wave.open(file_path)
                        sig = np.frombuffer(signal_wave.readframes(frame_rate), dtype=np.int16)
                        fig = plt.figure()
                        plt.specgram(sig, NFFT=1024, Fs=frame_rate, noverlap=900)
                        plt.axis('off')
                        fig.savefig(f'{file_dist_path}.png', dpi=fig.dpi)
                        plt.close()

In [2]:
BATCH_SIZE = 32
channels = 3
kernel = 4
stride = 1
pool = 2

In [3]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
output_dir="/content/drive/MyDrive/spectrograms_2"

In [4]:
#Make a dataset containing training spectrograms
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
                                             batch_size=BATCH_SIZE,
                                             validation_split=0.4,
                                             shuffle=True,
                                             color_mode='rgb',
                                             directory=output_dir,
                                             image_size=(432, 288),
                                             subset="training",
                                             seed=0)

# Make a dataset containing the validation spectrogram
valid_dataset = tf.keras.preprocessing.image_dataset_from_directory(
                                             batch_size=BATCH_SIZE,
                                             validation_split=0.4,
                                             shuffle=True,
                                             directory=output_dir,
                                             color_mode='rgb',
                                             image_size=(432, 288),
                                             subset="validation",
                                             seed=0)

Found 3000 files belonging to 10 classes.
Using 1800 files for training.
Found 3000 files belonging to 10 classes.
Using 1200 files for validation.


In [5]:
# split valid dataset to test and valid
test_ds = valid_dataset.take(8) # get 100 data from valid dataset
valid_dataset = valid_dataset.skip(8)# skip first 100

In [6]:
#ResNet50 Architecture
from tensorflow.keras.applications.resnet50 import ResNet50
base_model_2 = ResNet50(input_shape=(432,288,channels), weights='imagenet', include_top=False)
base_model_2.trainable = False #freeze the layers

94765736/94765736 [==============================] - 0s 0us/step


In [7]:
Model_4 = tf.keras.Sequential([
    base_model_2,
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10, activation='softmax')
])

In [8]:
Model_4.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), optimizer = tf.keras.optimizers.Adam(learning_rate = 0.0001), metrics = ['acc'])

In [10]:
History_4 = Model_4.fit(
train_dataset,
      epochs=2,
      validation_data=valid_dataset,  
      verbose=1)

Epoch 1/2
57/57 [==============================] - 295s 5s/step - loss: 1.4055 - acc: 0.6206 - val_loss: 0.5366 - val_acc: 0.8347
Epoch 2/2
57/57 [==============================] - 28s 473ms/step - loss: 0.2364 - acc: 0.9344 - val_loss: 0.3174 - val_acc: 0.9184


In [11]:
Model_4.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 14, 9, 2048)       23587712  
                                                                 
 flatten (Flatten)           (None, 258048)            0         
                                                                 
 dense (Dense)               (None, 10)                2580490   
                                                                 
Total params: 26,168,202
Trainable params: 2,580,490
Non-trainable params: 23,587,712
_________________________________________________________________


In [12]:
img = tf.keras.preprocessing.image.load_img("/content/drive/MyDrive/spectrograms_2/class_2/2_george_11.wav.png", target_size=(432, 288))
img_array = tf.keras.preprocessing.image.img_to_array(img)
print(img_array.shape)
img_array = tf.expand_dims(img_array, 0)
print(img_array.shape)
predictions = Model_4.predict(img_array)
score = tf.nn.softmax(predictions[0])
print( "Digit : "+ str(np.argmax(score)) +" ; Probability = "+ str(np.max(score)))
score

(432, 288, 3)
(1, 432, 288, 3)
1/1 [==============================] - 2s 2s/step
Digit : 2 ; Probability = 0.23007964


<tf.Tensor: shape=(10,), dtype=float32, numpy=
array([0.08547235, 0.08548091, 0.23007964, 0.08547381, 0.08567602,
       0.08563208, 0.08555143, 0.08546329, 0.08570482, 0.08546565],
      dtype=float32)>

In [15]:
np.argmax(score)

2

In [27]:
class_indices={'class_0': 0, 'class_1': 1, 'class_2': 2, 'class_3': 3, 'class_4': 4, 'class_5': 5, 'class_6': 6, 'class_7': 7, 'class_8': 8, 'class_9': 9}

In [32]:
Model_4.save('Spectogram.h5')

In [33]:
modelSaved = tf.keras.models.load_model('Spectogram.h5') 